# Installing, Cloning & Importing

In [ ]:
!pip install tensorflow-gpu==2.1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip /content/chest-xray-pneumonia.zip "*/train/*.*"

In [ ]:
!unzip /content/chest-xray-pneumonia.zip "*/test/*.*"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2, time
import tensorflow as tf

tf.__version__

# Loading The Datasets

In [ ]:
covid_path = '/content/gdrive/My Drive/Kaggle/pneumonia.csv'
covid_image_path = '/content/chest_xray/train/PNEUMONIA/'

normal_path = '/content/gdrive/My Drive/Kaggle/normal.csv'
normal_image_path = '/content/chest_xray/train/NORMAL/'

covid_df = pd.read_csv(covid_path, usecols=['filename', 'finding'])
normal_df = pd.read_csv(normal_path, usecols=['filename', 'finding'])

normal_df = normal_df.head(99)

covid_df.head()

In [ ]:
covid_images = []
covid_labels = []

for index, row in covid_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = covid_image_path + filename

    image = cv2.imread(path)
    image = cv2.resize(image,(224,224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    covid_images.append(image)
    covid_labels.append(label)

normal_images = []
normal_labels = []

for index, row in normal_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = normal_image_path + filename
    print(filename)
    # temporary fix while we preprocess ALL the images
    if filename == 'NORMAL2-IM-0381-0001.jpeg':
        break

    image = cv2.imread(path)
    image = cv2.resize(image,(224,224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    normal_images.append(image)
    normal_labels.append(label)

# normalize to interval of [0,1]
covid_images = np.array(covid_images) / 255

# normalize to interval of [0,1]
normal_images = np.array(normal_images) / 255

# Modeling With Keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten # Convert pools feature map into this large feature vector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

vggModel = VGG19(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

outputs = vggModel.output
outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation="softmax")(outputs)

model = Model(inputs=vggModel.input, outputs=outputs)

for layer in vggModel.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
)

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
# cross validation
# split into training and testing
n_splits=5
acc_scores = list()

X = np.concatenate(( normal_images, covid_images), axis=0)
Y = np.concatenate(( normal_labels, covid_labels), axis=0)
kf = KFold(n_splits=n_splits, random_state=1, shuffle=True)
kf.get_n_splits(X)

for fold, (train_index, test_index) in enumerate(kf.split(X)):
  print('\n Fold %d' % (fold))
  X_train,X_test=X[train_index],X[test_index]
  y_train,y_test=Y[train_index],Y[test_index]

 # make labels into categories - either 0 or 1
  y_train = LabelBinarizer().fit_transform(y_train)
  y_train = to_categorical(y_train)

  y_test = LabelBinarizer().fit_transform(y_test)
  y_test = to_categorical(y_test)
#  print(y_train)
  history = model.fit(train_aug.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    validation_steps=len(X_test) / 32,
                    steps_per_epoch=len(X_train) / 32,
                    epochs=100)
  acc = model.evaluate(X_test, y_test, verbose=0)
  acc_scores.append(acc[1])
  print('Fold %d: Accuracy %.2f%%' % (fold, acc[1] * 100))


In [ ]:
def average(numbers):
    """
    Return the sample arithmetic mean of data.
    :param numbers: a list of float
    :return:
    """
    try:
        return float(sum(numbers)) / max(len(numbers), 1)
    except ZeroDivisionError:
        return 0

In [ ]:
print('Accuracy scores: ', acc_scores)
mean_acc = average(acc_scores)
print('=====================')
print('Mean Accuracy %f' % mean_acc)
print('=====================')

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('classic')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('classic')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
acc = model.evaluate(X_test, y_test,verbose=0)
print("test acc:"+ str(acc[1]))

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('/content/chest_xray/test/PNEUMONIA/person1685_virus_2903.jpeg', target_size = (224,224))
#test_image = image.load_img('dataset/covid_adjusted/covid-19-pneumonia-rapidly-progressive-12-hours.jpg.jpg', target_size = (224,224))


test_image = image.img_to_array(test_image)
test_image = test_image *1./255

test_image = np.expand_dims(test_image, axis = 0)
test_image = np.vstack([test_image ])
result = model.predict(test_image,batch_size=32)
print(result)

In [ ]:
import matplotlib.image as mpimg

img=mpimg.imread('/content/chest_xray/test/PNEUMONIA/person1685_virus_2903.jpeg')
#img=mpimg.imread('dataset/covid_adjusted/covid-19-pneumonia-rapidly-progressive-12-hours.jpg.jpg')
imgplot = plt.imshow(img)
plt=plt.title('Chest X-ray ')

In [ ]:
if result[0][0]<result[0][1]:

    prediction = 'normal'
else:
    prediction = 'covid'
    
print("AI's prediction is: "+ prediction)